# Capstone: Evaluating Housing Program Effectiveness in Tooele County
### **An analysis of public housing program effectiveness** in meeting the needs of low- and moderate-income residents in Tooele County, Utah.
### - Research Question:
> "Will the projected housing prices in Tooele County, Utah, in 2025 increase at a greater rate than the projected income growth of local residents?"
### - Hypothesis:
> "_Projected housing prices_ of available housing stock in 2025 __will increase at a greater rate__ than the _projected income_ of residents in Tooele County, Utah."

In [ ]:
# Import libraries
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import requests
import json
from datetime import datetime
import kagglehub
import glob

## 1. Introduction
- **Research Question:** How effective are housing assistance programs in Tooele County at supporting low- and moderate-income families?
- **Focus Areas:** Section 8, Family Self-Sufficiency, Permanent Supportive Housing, Homeownership Programs

## 📚 1.2. Background and Literature Review

I. Housing Affordability: 2024 Utah Priorities Project  
   https://www.utahfoundation.org/reports/housing-affordability-2024-priorities-brief-1/

II. American Community Survey (ACS) – U.S. Census Bureau  
   https://www.census.gov/programs-surveys/acs/

III. National Association of Realtors Housing Forecast  
   https://www.nar.realtor/research-and-statistics/housing-statistics


## 2. Data Acquisition

* ### API Calls and Static Data Downloads
> - **Income Data** [income_df]: U.S. Census Bureau (ACS API)
> - **Housing Prices** [housing_df]: Zillow ZHVI via Kaggle or scraped CSV
> - **Employment Data** [bls_df]: BLS API
> - **Local Economics** [local_df]: Utah DWS

* ### Data Cleaning & Preparation
> - Drop nulls, standardize formats, join datasets
> - Feature engineering: affordability ratios, utilization rates


### 2.1 Income Data
---

In [ ]:

CENSUS_API_KEY = "de53f7491b5574a451233a6d04721c4176263ab4"
years = range(2015, 2023)  # 2023 ACS not fully available yet
results = []

for year in years:
    url = (
        f"https://api.census.gov/data/{year}/acs/acs5"
        f"?get=NAME,B19013_001E&for=county:045&in=state:49&key={CENSUS_API_KEY}"
    )
    response = requests.get(url)
    if response.status_code == 200:
        try:
            data = response.json()
            value = data[1][1]
            results.append({"year": year, "median_income": int(value)})
        except Exception as e:
            print(f"Error parsing {year}: {e}")
    else:
        print(f"Failed for {year}: {response.status_code}")

# Convert to DataFrame
income_df = pd.DataFrame(results)
income_df = income_df.sort_values("year")
income_df['monthly_income'] = income_df["median_income"] / 12
income_df.head()

### 2.2 Housing Prices
#### 2.2.1 Zillow Home Value Index
---

In [ ]:
# Housing Prices – Zillow ZHVI (Zillow Home Value Index)

path = kagglehub.dataset_download("robikscube/zillow-home-value-index")
# print("Path to dataset files:", path)

# List all downloaded files to verify filenames
# for root, dirs, files in os.walk(path):
    # for file in files:
        # print(os.path.join(root, file))

# Load all home price index data
zhvi_path = os.path.join(path, "ZHVI.csv")  # Adjust if the filename is different
zhvi_df = pd.read_csv(zhvi_path)

zhvi_df.rename(columns={zhvi_df.columns[0]: "date"}, inplace=True)

# Filter for Utah
zhvi_ut_df = zhvi_df[['date', 'Utah']].copy()
# Format 'date' to datetime and normalize dates to first of the month
zhvi_ut_df['date'] = pd.to_datetime(zhvi_ut_df['date']).dt.to_period('M').dt.to_timestamp()
zhvi_ut_df.head()

In [ ]:
# Join Utah ZHVI data with Tooele and Salt Lake County Data

# County data is reported on the last day of the month and State data reported on the first, 
# dates will be normalized to the first of the month, with county data shifted forward a day

zhvi_county_df = pd.read_csv("data/zhvi_county_data.csv")
# Format 'date' to datetime and normalize dates to first of the month
zhvi_county_df['date'] = pd.to_datetime(zhvi_county_df['date']).dt.to_period('M').dt.to_timestamp()
# Shift month forward by 1
zhvi_county_df['date'] = zhvi_county_df['date'] + pd.DateOffset(months=1)

zhvi_all_df = pd.merge(zhvi_ut_df, zhvi_county_df, on='date', how='outer')

housing_df = zhvi_all_df[zhvi_all_df['date'] >= '2015-01-01']

# housing_df.info()
housing_df.head(10)
# housing_df.tail(10)

#### 2.2.2 Zillow Observed Rent Index
---

In [ ]:
# Download and load the Zillow Rent Index dataset
path = kagglehub.dataset_download("zillow/rent-index")
rent_index_path = os.path.join(path, "price.csv")
rent_index_df = pd.read_csv(rent_index_path)

# Filter for Tooele County
rent_index_df = rent_index_df[rent_index_df['County'] == "Tooele"]

# Drop unneeded columns
columns_to_drop = ["City Code", "Metro", "State", "Population Rank"]
rent_index_df.drop(columns=columns_to_drop, inplace=True)

# Set 'City' as the index and transpose the date columns
rent_index_df.set_index('City', inplace=True)

# Transpose the DataFrame
rent_df1 = rent_index_df.T.copy()

# Convert index to datetime if it's month strings like "2020-01"
rent_df1.index = pd.to_datetime(rent_df1.index, errors='coerce')

# Clean up any rows with invalid dates (if any)
rent_df1 = rent_df1[rent_df1.index.notnull()]

# Add a County mean summary column
rent_df1['Tooele County'] = rent_df1.mean(axis=1)

rent_df1 = rent_df1.dropna(subset=["Tooele County"])

rent_df1 = rent_df1.reset_index().rename(columns={'index': 'date'})
rent_df1.index.name = None
rent_df1 = rent_df1[(rent_df1['date'] >= '2015-01-01') & (rent_df1['date'] < '2017-01-01')]

rent_df1.tail()

In [ ]:
# Load the CSV file from the 'data' directory
csv_path = os.path.join("data", "county_zori_data.csv")
rent_index_df2 = pd.read_csv(csv_path)

# Filter for Tooele County
rent_index_df2 = rent_index_df2[rent_index_df2['RegionName'] == "Tooele County"]
rent_index_df2.head()



# Drop unneeded columns
columns_to_drop = ["RegionID", "RegionType", "StateName", "State", "Metro", "SizeRank", "StateCodeFIPS", "MunicipalCodeFIPS"]
rent_index_df2.drop(columns=columns_to_drop, inplace=True)

# Set 'City' as the index and transpose the date columns
rent_index_df2.set_index('RegionName', inplace=True)
rent_df2 = rent_index_df2.T.copy()

# Convert index to datetime if it's month strings like "2020-01"
rent_df2.index = pd.to_datetime(rent_df2.index, errors='coerce')

# Clean up any rows with invalid dates (if any)
rent_df2 = rent_df2[rent_df2.index.notnull()]

# Drop rows where the summary column is NaN
rent_df2 = rent_df2.dropna(subset=["Tooele County"])

# Reset index and rename
rent_df2 = rent_df2.reset_index().rename(columns={'index': 'date'})
rent_df2.index.name = None

# Format 'date' to datetime and normalize dates to first of the month
rent_df2['date'] = pd.to_datetime(rent_df2['date']).dt.to_period('M').dt.to_timestamp()
# Shift month forward by 1
rent_df2['date'] = rent_df2['date'] + pd.DateOffset(months=1)

# Filter for dates starting from January 2015
rent_df2 = rent_df2[rent_df2['date'] >= '2015-01-01']

rent_df2.head()

In [ ]:
# Select just the date and Tooele County columns
df1 = rent_df1[['date', 'Tooele County']].copy()
df2 = rent_df2[['date', 'Tooele County']].copy()

# Concatenate the two DataFrames
combined_df = pd.concat([df1, df2])

# Set 'date' as the index
combined_df.set_index('date', inplace=True)

#  Create a complete monthly date range
full_index = pd.date_range(start=combined_df.index.min(), end=combined_df.index.max(), freq='MS')

# Reindex and interpolate missing months
combined_df = combined_df.reindex(full_index)
combined_df.index.name = 'date'

# Interpolate missing rent values linearly
# combined_df['Tooele County'] = combined_df['Tooele County'].interpolate(method='linear', limit_direction='both')
# combined_df['Tooele County'] = combined_df['Tooele County'].fillna(method='ffill').fillna(method='bfill')
# Interpolate and fill edge NaNs
# combined_df['Tooele County'] = (
    # combined_df['Tooele County']
    # .interpolate(method='linear', limit_direction='both')
    # .fillna(method='ffill')
    # .fillna(method='bfill')
# )

# Reset index if needed
rent_df = combined_df.reset_index()
rent_df.rename(columns={'index': 'date'}, inplace=True)

rent_df.tail(50)

### 2.3 Employment Data
---

In [ ]:
# Employment Data: Bureau of Labor Statistics (BLS) API

# JSON data request
headers = {'Content-type': 'application/json'}
data = json.dumps({
  "seriesid": ["LAUCN490450000000003"],
  "startyear": "2015",
  "endyear": "2024",
  "registrationkey": "6563620fb1c04af6ab1809ebe75012e2"
})

response = requests.post('https://api.bls.gov/publicAPI/v2/timeseries/data/', data=data, headers=headers)
bls_data = response.json()

# Convert to DataFrame
series_data = bls_data['Results']['series'][0]['data']
bls_df = pd.DataFrame(series_data)

# Clean and format data

bls_df = bls_df[bls_df['period'].str.startswith('M')]                                       # Keep only month periods (ignore annual summaries or irregular codes)
bls_df['date'] = pd.to_datetime(bls_df['year'] + '-' + bls_df['period'].str[1:] + '-01')    # Convert to datetime
bls_df['value'] = pd.to_numeric(bls_df['value'])                                            # Convert 'value' to numeric (it's a string by default)
bls_df.rename(columns={'value': 'UnemploymentRate'}, inplace=True)                          # Rename for clarity
bls_df = bls_df[['date', 'UnemploymentRate', 'year']]                                       # Reduce to desired columns
bls_df = bls_df.sort_values('date').reset_index(drop=True)                                  # Sort by date

bls_df.head()

### 2.4 Local Economics Data
---

In [ ]:
# Local Economics – Utah Department of Workforce Services (DWS)

# Set the path to your data folder
data_dir = 'data'
pattern = os.path.join(data_dir, 'industry-and-wages-data-*.csv')

# Collect all matching file paths
file_paths = glob.glob(pattern)

# Read and combine all CSVs, adding a 'Year' column extracted from the filename
df_list = []
for file_path in file_paths:
    year = int(os.path.basename(file_path).split('-')[-1].split('.')[0])  # Extract year from filename
    df = pd.read_csv(file_path)
    df['Year'] = year
    df_list.append(df)

# Merge into a single DataFrame
local_df = pd.concat(df_list, ignore_index=True)

# Optional: Sort by Year
local_df.sort_values(by='Year', inplace=True)

# View result
# print(local_df.head())
local_df.head(10)

## 3. Exploratory Data Analysis (EDA)
- Trends in rent, income, voucher use
- Maps and charts

In [ ]:
# Create view, housing prices grouped by annual mean, with estimated mean down payment
housing_mn_df = housing_df.copy()
housing_mn_df['year'] = housing_mn_df['date'].dt.year
housing_mn_df = housing_mn_df.groupby('year', as_index=False)['Tooele County'].mean()
housing_mn_df.rename(columns={'Tooele County': 'median_price'}, inplace=True)
housing_mn_df['down_payment'] = housing_mn_df['median_price'] * 0.15

housing_mn_df.head()

In [ ]:
# Merge housing and income data on 'year'
merged_df = pd.merge(housing_mn_df, income_df, on='year', how='inner')

# Plot
plt.figure(figsize=(12, 6))

# Plot median housing price
sns.lineplot(data=merged_df, x="year", y="median_price", label="Median Housing Price", color="blue")
for i, row in merged_df.iterrows():
    plt.text(row["year"], row["median_price"] + 500, f"{row['median_price']:.0f}", color='blue', fontsize=8, ha='center')

# Plot down payment
sns.lineplot(data=merged_df, x="year", y="down_payment", label="Down Payment (15%)", color="green")
for i, row in merged_df.iterrows():
    plt.text(row["year"], row["down_payment"] + 500, f"{row['down_payment']:.0f}", color='green', fontsize=8, ha='center')

# Plot median income
sns.lineplot(data=merged_df, x="year", y="median_income", label="Median Annual Income", color="orange")
for i, row in merged_df.iterrows():
    plt.text(row["year"], row["median_income"] + 500, f"{row['median_income']:.0f}", color='orange', fontsize=8, ha='center')

# Formatting
plt.title("Median Housing Price, Down Payment, and Income in Tooele County")
plt.xlabel("Year")
plt.ylabel("USD")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
# Create view, rent prices grouped by annual mean
rent_mn_df = rent_df.copy()
rent_mn_df['year'] = rent_mn_df['date'].dt.year
rent_mn_df = rent_mn_df.groupby('year', as_index=False)['Tooele County'].mean()
rent_mn_df.rename(columns={'Tooele County': 'median_price'}, inplace=True)
rent_mn_df['median_price'] = rent_mn_df['median_price'].interpolate(method='linear', limit_direction='both')

rent_mn_df#.head()

In [ ]:
# # Line plot of historical trends
# plt.figure(figsize=(12, 6))
# sns.lineplot(data=rent_mn_df, x="year", y="median_price", label="Housing Price")
# sns.lineplot(data=income_df, x="year", y="median_income", label="Income")
# plt.title("Historical Trends: Median Housing Prices vs. Income (Tooele County)")
# plt.ylabel("USD")
# plt.xlabel("Year")
# plt.legend()
# plt.grid(True)
# plt.show()


# Merge housing and income data on 'year'
merged_df2 = pd.merge(rent_mn_df, income_df, on='year', how='inner')

# Plot
plt.figure(figsize=(12, 6))

# Plot median housing price
sns.lineplot(data=merged_df2, x="year", y="median_price", label="Median Housing Price", color="blue")
for i, row in merged_df2.iterrows():
    plt.text(row["year"], row["median_price"] + 500, f"{row['median_price']:.0f}", color='blue', fontsize=8, ha='center')

# Plot median income
sns.lineplot(data=merged_df2, x="year", y="monthly_income", label="Median Annual Income", color="orange")
for i, row in merged_df2.iterrows():
    plt.text(row["year"], row["monthly_income"] + 500, f"{row['monthly_income']:.0f}", color='orange', fontsize=8, ha='center')

# Formatting
plt.title("Median Housing Price, Down Payment, and Income in Tooele County")
plt.xlabel("Year")
plt.ylabel("USD")
plt.grid(True)
plt.legend()
plt.tight_layout()
plt.show()

## 5. Evaluation Metrics
- Rent-to-income ratio

## 6. Conclusions and Recommendations
- Summary of findings
- Policy insights

## 7. Code Appendix
- Functions and utilities